In [1]:
import datasets
import glob
from pathlib import Path
import sys
import os
from tqdm import tqdm

sys.path.append('../../')

from examination_paper_classifier import extract_text

In [2]:
# 存一个序列，用于转化dataset，这里简单的用了dict，也可以pandas等
train_dataset_dict = {"text":[], "label":[], "file_path":[]}

In [4]:
# 如果之前存在dataset
datadict = datasets.load_from_disk("../dataset")
train_dataset = datadict["train"]
train_dataset

Dataset({
    features: ['text', 'label', 'file_path'],
    num_rows: 312
})

In [6]:
exists_file_path = {row["file_path"] for row in train_dataset}
len(exists_file_path)

312

In [7]:
train_examination_paper_file_folder = Path("../negative_file")
all_negative_files = train_examination_paper_file_folder.glob("**/*.doc")

for file_path in tqdm(list(all_negative_files)):
    if str(file_path) in exists_file_path:
        continue
        
    try:
        ext = file_path.suffix
        text = extract_text(str(file_path), ext)

        if text:
            train_dataset_dict["text"].append(text)
            train_dataset_dict["label"].append(1)
            train_dataset_dict["file_path"].append(str(file_path))
    except Exception as e:
        # print(f"Error processing file {file_path}: {e}")
        continue

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:10<00:00, 24.31it/s]


In [8]:
train_not_examination_paper_file_folder = Path("../positive_file")
all_positive_files = train_not_examination_paper_file_folder.glob("**/*.doc")

for file_path in tqdm(list(all_positive_files)):
    if str(file_path) in exists_file_path:
        continue
        
    try:
        ext = file_path.suffix
        text = extract_text(str(file_path), ext)

        if text:
            train_dataset_dict["text"].append(text)
            train_dataset_dict["label"].append(0)
            train_dataset_dict["file_path"].append(str(file_path))
    except Exception as e:
        continue

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 85/85 [00:01<00:00, 46.11it/s]


In [9]:
train_dataset = datasets.Dataset.from_dict(train_dataset_dict)
train_dataset

Dataset({
    features: ['text', 'label', 'file_path'],
    num_rows: 312
})

In [10]:
datadict = datasets.DatasetDict({
    "train": train_dataset
})

In [11]:
datadict.save_to_disk("./dataset")

Saving the dataset (0/1 shards):   0%|          | 0/312 [00:00<?, ? examples/s]